In [1]:
#Notebook showing how to execute annotation with SDG with a custom annotation yaml

# Annotation with SDG

## Importing the necessary libraries

In [2]:
# First Party
from instructlab.sdg.pipeline import Pipeline, PipelineContext
# Third Party
from datasets import load_dataset
from openai import OpenAI
import yaml
import os

/home/ec2-user/subsetenv/lib64/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Serve LLM through ilab serve command

Run the following shell command to serve the Mixtral-8x7B-Instruct-v0.1 model on port 8000 (by default). The mixtral model is quite large and may take a while to be served through vLLM.

*Note*: You can serve any other desired model by changing the model-path argument. The rest of this notebook will work seamlessly with any other model as long we can wrap the served model in an OpenAI client

`ilab serve --model-path ~/.cache/instructlab/models/mistralai/Mixtral-8x7B-Instruct-v0.1/`

Wrap the served model in an OpenAI client

In [3]:
client = OpenAI(
    base_url="http://localhost:8000/v1",  # Your model endpoint
    api_key="dummy-key"  # vLLM doesn't check the key, but one is required
)

Make sure the model is served before running the next cell, and that the following cell returns the correct model id

In [4]:
models = client.models.list()
teacher_model = models.data[0].id
teacher_model #make sure this is the correct model

'/home/ec2-user/.cache/instructlab/models/mistralai/Mixtral-8x7B-Instruct-v0.1'

# Preparing classification dataset

#### In this exercise, we will use the Yahoo Answers Topics dataset from HuggingFace.

#### Here, we will use a small portion of the training set to demonstrate the prompt engineering process, and the rest of the set will be assumed to be unlabeled.


Steps to follow:
1. EDA of the dataset
2. Create In-Context Learning examples
3. Iterate on the annotation pipeline (the components of the prompt) to improve the quality of the annotation
4. Merge the ICL examples, unlabeled samples and the components of the prompt into a final input dataset for annotation

## Importing classification dataset from HuggingFace

In [5]:
# Importing classification dataset from HuggingFace
dataset = load_dataset("fancyzhx/ag_news")
print(dataset) # print details of the dataset

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 120000
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 7600
    })
})


Let us use a portion of the dataset for this example, since the dataset is quite large (1.4 million samples)

In [6]:
#randomly select 500 samples
dataset = dataset['train'].shuffle(seed=42).select(range(500))
print(dataset) # print details of the dataset

Dataset({
    features: ['text', 'label'],
    num_rows: 500
})


## EDA of the dataset

In [7]:
# After loading the dataset, add these EDA steps
import pandas as pd
from collections import Counter
import statistics

# Convert Dataset to pandas DataFrame for easier analysis
df = pd.DataFrame(dataset)
labels = dataset.features['label'].names
# 1. Basic Dataset Information
print("\n=== Dataset Overview ===")
print(f"Total number of samples: {len(df)}")
print("\nFeature Information:")
print(df.info())

# 2. Class Distribution
print("\n=== Class Distribution ===")
class_dist = df['label'].map(lambda x: labels[x]).value_counts()
for class_name, count in class_dist.items():
    percentage = (count/len(df)) * 100
    print(f"{class_name}: {count} samples ({percentage:.1f}%)")

# 3. Text Length Analysis
df['text_length'] = df['text'].str.len()
print("\n=== Text Length Statistics ===")
print(df['text_length'].describe())

# 4. Text Length by Class
print("\n=== Average Text Length by Class ===")
for label_idx, label_name in enumerate(labels):
    class_texts = df[df['label'] == label_idx]['text_length']
    print(f"{label_name}:")
    print(f"  Mean length: {class_texts.mean():.1f} characters")
    print(f"  Min length: {class_texts.min()} characters")
    print(f"  Max length: {class_texts.max()} characters")

# 5. Word Count Analysis
df['word_count'] = df['text'].str.split().str.len()
print("\n=== Word Count Statistics ===")
print(df['word_count'].describe())

# 6. Word Count by Class
print("\n=== Average Word Count by Class ===")
for label_idx, label_name in enumerate(labels):
    class_words = df[df['label'] == label_idx]['word_count']
    print(f"{label_name}:")
    print(f"  Mean words: {class_words.mean():.1f}")
    print(f"  Min words: {class_words.min()}")
    print(f"  Max words: {class_words.max()}")

# 7. Most Common Words
def get_words(text):
    words = text.lower().split()
    return [word for word in words if word.isalnum() and len(word) > 3]  # Simple filtering

all_words = []
for text in df['text']:
    all_words.extend(get_words(text))

print("\n=== Most Common Words ===")
word_freq = Counter(all_words).most_common(20)
print("Top 20 most frequent words:")
for word, count in word_freq:
    print(f"{word}: {count}")

# 8. Sample texts from each class
print("\n=== Sample Text from Each Class ===")
for label_idx, label_name in enumerate(labels):
    sample_text = df[df['label'] == label_idx]['text'].iloc[0]
    print(f"\n{label_name.upper()}:")
    print(sample_text[:200] + "..." if len(sample_text) > 200 else sample_text)

# 9. Basic Statistics Summary
print("\n=== Basic Statistics Summary ===")
print(f"Number of unique documents: {df['text'].nunique()}")
print(f"Average words per document: {df['word_count'].mean():.1f}")
print(f"Median words per document: {df['word_count'].median()}")
print(f"Most common class: {class_dist.index[0]} ({class_dist.iloc[0]} samples)")
print(f"Least common class: {class_dist.index[-1]} ({class_dist.iloc[-1]} samples)")


=== Dataset Overview ===
Total number of samples: 500

Feature Information:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 500 entries, 0 to 499
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   text    500 non-null    object
 1   label   500 non-null    int64 
dtypes: int64(1), object(1)
memory usage: 7.9+ KB
None

=== Class Distribution ===
Sci/Tech: 137 samples (27.4%)
Sports: 135 samples (27.0%)
World: 114 samples (22.8%)
Business: 114 samples (22.8%)

=== Text Length Statistics ===
count    500.000000
mean     231.006000
std       64.919265
min      107.000000
25%      191.000000
50%      226.500000
75%      263.000000
max      801.000000
Name: text_length, dtype: float64

=== Average Text Length by Class ===
World:
  Mean length: 232.8 characters
  Min length: 130 characters
  Max length: 488 characters
Sports:
  Mean length: 224.5 characters
  Min length: 116 characters
  Max length: 801 characters
Business:
  Mean l

## Creating In-Context-Learning examples (Few-Shot Examples) for the Prompt

Let us first select 3 examples from the dataset to be used as In-Context-Learning examples (Few-Shot Examples) for the prompt, and 20 examples to be used as validation examples for prompt engineering. The rest, we will save for labeling by the annotation pipeline.


In [8]:
K = 3 #number of ICL examples
N = 30 #number of validation examples to be used for prompt engineering
icl_samples = dataset.select(range(K))
validation_samples = dataset.select(range(K, K+N))
unlabeled_samples = dataset.select(range(K+N, len(dataset)))

print(f"ICL examples: {len(icl_samples)}")
print(f"Validation examples: {len(validation_samples)}")
print(f"Unlabeled examples: {len(unlabeled_samples)}")

ICL examples: 3
Validation examples: 30
Unlabeled examples: 467


In [9]:
for sample in icl_samples:
    print("\n", sample['text'], "\nLabel: ", labels[int(sample['label'])])


 Bangladesh paralysed by strikes Opposition activists have brought many towns and cities in Bangladesh to a halt, the day after 18 people died in explosions at a political rally. 
Label:  World

 Desiring Stability Redskins coach Joe Gibbs expects few major personnel changes in the offseason and wants to instill a culture of stability in Washington. 
Label:  Sports

 Will Putin #39;s Power Play Make Russia Safer? Outwardly, Russia has not changed since the barrage of terrorist attacks that culminated in the school massacre in Beslan on Sept. 
Label:  World


These look like good examples to be used as few shot examples for the prompt. We will save this for now

## Prompt Engineering

In this section, we will iterate on the prompt to improve the quality of the annotation.

- We will start with a basic prompt and then iterate on it based on the performance on the validation examples.
- We will cover 3 iterations of prompt engineering:
    - Basic prompt
    - Prompt with ICL examples and structured principles and system prompt
    - Prompt with improved ICL examples

Create annotation config YAML with 6 prompt components:
- System prompt (the overall instruction for the task. empty for now)
- Introduction (a brief introduction to the task)
- Principles (the principles that guide the task, empty for now)
- Examples (empty for now)
- Generation (the query for annotation, along with any prefix or suffix instructions)
- Start tags and end tags (empty)

#### Note the templating pattern for the introduction and generation components. These will be used to inject the values for the task description and the query for annotation respectively, dynamically. We need to make sure that these keys are present in the input dataset, when we call `pipeline.generate()`. The keys used in this example are `simple_task_description` and `text`. You can use any other keys that you want to inject into the prompt, but these should be present in the input dataset.

In [10]:
# Create annotation config YAML
simple_annotation_config = {
    "system": None,
    "introduction": "Task Description: {{simple_task_description}}",
    "principles": None,
    "examples": None,
    "generation": "Here is the query for annotation:\n{{text}}",
    "start_tags": [""],
    "end_tags": [""]
}

# Write to YAML file
with open('simple_annotation_config.yaml', 'w') as f:
    yaml.dump(simple_annotation_config, f, default_flow_style=False)

In [11]:
#Let's create 'simple_task_description' key in the validation_samples dataset and populate it with the task description.
simple_task_description = "Annotation"
validation_samples = validation_samples.map(lambda x: {"simple_task_description": simple_task_description})
validation_samples

Dataset({
    features: ['text', 'label', 'simple_task_description'],
    num_rows: 30
})

Create annotation yaml configuration to leverage guided decoding, and include the labels under the 'guided choice' key like so. We are going to make this point to the simple_annotation_config.yaml file that we just created above.

In [15]:
# Create YAML configuration
yaml_config = {
    "version": "1.0",
    "blocks": [
        {
            "name": "annotation",
            "type": "LLMBlock",
            "config": {
                "config_path": "simple_annotation_config.yaml",
                "model_id": "mistralai/Mixtral-8x7B-Instruct-v0.1",
                "output_cols": ["output"],
                "gen_kwargs": {
                    "max_tokens": 20,
                    "temperature": 0,
                    "extra_body": {
                        "guided_decoding_backend": "xgrammar", #use xgrammar backend for guided decoding, explicitly, and only xgrammar with no fallback on error
                        "guided_choice": labels  # This will use your labels list
                    }
                }
            },
            "drop_duplicates": ["text"]
        }
    ]
}

# Write to YAML file
with open('annotation_pipeline.yaml', 'w') as f: #this is the file that will be used to create the annotation pipeline
    yaml.dump(yaml_config, f, default_flow_style=False)

### Initialize pipeline context and annotation pipeline

In [16]:
ctx = PipelineContext(client=client, model_family="mixtral", model_id=teacher_model)
# constructing the path with the 'annotation' directory explicitly
current_dir = os.path.dirname(os.path.abspath(''))
pipeline_yaml = os.path.join(current_dir, "annotation", "annotation_pipeline.yaml")
annotation_pipe = Pipeline.from_file(ctx, pipeline_yaml)

Main Driver Code

In [17]:
gen_data = annotation_pipe.generate(validation_samples)

Check output features

In [18]:
gen_data.features

{'text': Value(dtype='string', id=None),
 'label': Value(dtype='int64', id=None),
 'simple_task_description': Value(dtype='string', id=None),
 'output': Value(dtype='string', id=None)}

Print generated samples with true and predicted labels

In [19]:
for sample in gen_data:
    print("\ntext: ", sample['text'], "\ntrue label: ", labels[int(sample['label'])], "\npredicted label: ", sample['output'])



text:  U2 pitches for Apple New iTunes ads airing during baseball games Tuesday will feature the advertising-shy Irish rockers. 
true label:  Sci/Tech 
predicted label:  Sports

text:  S African TV in beheading blunder Public broadcaster SABC apologises after news bulletin shows footage of American beheaded in Iraq. 
true label:  World 
predicted label:  Sports

text:  A Cosmic Storm: When Galaxy Clusters Collide Astronomers have found what they are calling the perfect cosmic storm, a galaxy cluster pile-up so powerful its energy output is second only to the Big Bang. 
true label:  Sci/Tech 
predicted label:  Sci/Tech

text:  West sets deadline for Iran to freeze uranium enrichment Four western countries set the scene yesterday for a showdown with Iran by demanding that it freeze its uranium enrichment activities immediately. 
true label:  World 
predicted label:  Sports

text:  Computer Assoc. Cuts 800 Jobs Worldwide (AP) AP - Computer Associates International Inc. announced a restru

We can see that the predicted labels are not very good. Let's check the performance of the pipeline on the validation examples.


In [20]:
#accuracy metrics
from sklearn.metrics import accuracy_score

# Calculate basic accuracy
true_labels = list(map(lambda x: labels[int(x['label'])], gen_data))
pred_labels = list(map(lambda x: x['output'], gen_data))
accuracy = accuracy_score(true_labels, pred_labels)
print(f"Accuracy: {accuracy:.2%}")

Accuracy: 30.00%


In [21]:
print(true_labels)
print(pred_labels)

['Sci/Tech', 'World', 'Sci/Tech', 'World', 'Sci/Tech', 'Sci/Tech', 'Business', 'Sports', 'Business', 'Sci/Tech', 'World', 'Business', 'Sports', 'Sports', 'Business', 'Business', 'Sci/Tech', 'Sci/Tech', 'Business', 'Sci/Tech', 'Business', 'World', 'Sports', 'Sci/Tech', 'Business', 'Sports', 'Sports', 'Sci/Tech', 'Sports', 'World']
['Sports', 'Sports', 'Sci/Tech', 'Sports', 'Sports', 'Business', 'Sports', 'Sports', 'Business', 'Sports', 'Sports', 'Sports', 'Sports', 'Sports', 'Sports', 'Sports', 'Business', 'Sports', 'Sports', 'Business', 'Sports', 'Sports', 'Sports', 'Business', 'Sports', 'Sports', 'Sports', 'Sports', 'Sports', 'Sports']



Let's iterate on the prompt to improve the quality of the annotation.
Now we will create a new annotation pipeline config which uses:

- the ICL examples to improve the quality of the annotation.
- the principles to guide the annotation.
- the system prompt to provide overall instructions for the task.

Let's start by creating the principles and the system prompt for annotation.

In [22]:
task_description = "annotate the following text with the appropriate category based on the context of the text."

principles = """Important guidelines for classification:
- Focus on the main topic, not peripheral mentions
- Look for specific keywords that indicate the category
- Choose the most specific applicable category
- Be consistent with similar types of questions"""

system_prompt = """
You are an expert in annotation. You will be given a text and you need to annotate it with the appropriate category based on the context of the text.
"""


Now we will prepare dataset to include the ICLs and the principles and the system prompt for annotation, in each row

In [23]:
# Prepare your dataset with all template variables
validation_samples = validation_samples.map(lambda x: {
    "simple_task_description": task_description,
    "principles": principles,
    "system_prompt": system_prompt,
    "questions_and_answers": [
        {
            "question": icl_samples[0]["text"],
            "answer": labels[int(icl_samples[0]["label"])]
        },
        {
            "question": icl_samples[1]["text"],
            "answer": labels[int(icl_samples[1]["label"])]
        },
        {
            "question": icl_samples[2]["text"],
            "answer": labels[int(icl_samples[2]["label"])]
        }
    ]
})

check features of dataset to make sure everything needed for the template is present

In [24]:
validation_samples.features

{'text': Value(dtype='string', id=None),
 'label': ClassLabel(names=['World', 'Sports', 'Business', 'Sci/Tech'], id=None),
 'simple_task_description': Value(dtype='string', id=None),
 'principles': Value(dtype='string', id=None),
 'system_prompt': Value(dtype='string', id=None),
 'questions_and_answers': [{'answer': Value(dtype='string', id=None),
   'question': Value(dtype='string', id=None)}]}

In [25]:

detailed_annotation_config = {
    "system": "{{system_prompt}}",
    "introduction": "Task Description: {{ simple_task_description }}",
    "principles": "{{ principles }}",
    "examples": """To better assist you with this task, here are some examples:
{% if questions_and_answers is defined %}
{% for sample in questions_and_answers %}
[Start of Question]
{{ sample.question }}
[End of Question]

[Start of Output]
{{ sample.answer }}
[End of Output]
{% endfor %}
{% else %}
[Start of Question]
{{ seed_question }}
[End of Question]

[Start of Output]
{{ seed_response }}
[End of Output]
{% endif %}""",
    "generation": """Here is the query for annotation:
  [Start of Question]
  {{text}}
  [End of Question]""",
    "start_tags": [""],
    "end_tags": [""]
}

with open('detailed_annotation_config.yaml', 'w') as f:
    yaml.dump(detailed_annotation_config, f, default_flow_style=False)

Make sure that the `annotation_pipeline.yaml` file is pointing to the `detailed_annotation_config.yaml` file now.

In [26]:
annotation_pipeline_yaml = yaml.safe_load(open('annotation_pipeline.yaml'))
annotation_pipeline_yaml['blocks'][0]['config']['config_path'] = 'detailed_annotation_config.yaml'
with open('annotation_pipeline.yaml', 'w') as f:
    yaml.dump(annotation_pipeline_yaml, f, default_flow_style=False)


Run the pipeline again with the new configs, and check the results.

In [27]:
ctx = PipelineContext(client=client, model_family="mixtral", model_id=teacher_model)
# constructing the path with the 'annotation' directory explicitly
current_dir = os.path.dirname(os.path.abspath(''))
pipeline_yaml = os.path.join(current_dir, "annotation", "annotation_pipeline.yaml")

annotation_pipe = Pipeline.from_file(ctx, pipeline_yaml)

gen_data = annotation_pipe.generate(validation_samples)
for sample in gen_data:
    print("\ntext: ", sample['text'], "\ntrue label: ", labels[int(sample['label'])], "\npredicted label: ", sample['output'])


text:  U2 pitches for Apple New iTunes ads airing during baseball games Tuesday will feature the advertising-shy Irish rockers. 
true label:  Sci/Tech 
predicted label:  Sports

text:  S African TV in beheading blunder Public broadcaster SABC apologises after news bulletin shows footage of American beheaded in Iraq. 
true label:  World 
predicted label:  World

text:  A Cosmic Storm: When Galaxy Clusters Collide Astronomers have found what they are calling the perfect cosmic storm, a galaxy cluster pile-up so powerful its energy output is second only to the Big Bang. 
true label:  Sci/Tech 
predicted label:  Sci/Tech

text:  West sets deadline for Iran to freeze uranium enrichment Four western countries set the scene yesterday for a showdown with Iran by demanding that it freeze its uranium enrichment activities immediately. 
true label:  World 
predicted label:  World

text:  Computer Assoc. Cuts 800 Jobs Worldwide (AP) AP - Computer Associates International Inc. announced a restruct

In [28]:
# Calculate basic accuracy
true_labels = list(map(lambda x: labels[int(x['label'])], gen_data))
pred_labels = list(map(lambda x: x['output'], gen_data))
accuracy = accuracy_score(true_labels, pred_labels)
print(f"Accuracy: {accuracy:.2%}")

Accuracy: 60.00%


## Improve the quality of the ICL examples

We have improved the accuracy but not enough. Let's try to improve the quality of the ICL examples, by creating at least one ICL example for each of the labels so the model can learn to annotate all the labels correctly.

In [29]:
# First, get one example for each label
icl_examples = []
seen_labels = set()
used_indices = set()  # Keep track of which indices we've used

# Iterate through the dataset until we have an example for each label
for idx, sample in enumerate(unlabeled_samples):
    label_idx = int(sample['label'])
    label_name = labels[label_idx]

    # If we haven't seen this label yet, add it to our examples
    if label_name not in seen_labels:
        icl_examples.append({
            "question": sample["text"],
            "answer": label_name
        })
        seen_labels.add(label_name)
        used_indices.add(idx)

    # Break if we have all labels
    if len(seen_labels) == len(labels):
        break

# Remove the used examples from unlabeled_samples
# We need to convert to list and sort in descending order to avoid index shifting
remaining_indices = [i for i in range(len(unlabeled_samples)) if i not in used_indices]
unlabeled_samples = unlabeled_samples.select(remaining_indices)

# Verify the results
print("Number of ICL examples:", len(icl_examples))
print("Labels covered:", sorted(list(seen_labels)))
print("Missing labels:", set(labels) - seen_labels)
print("Original unlabeled samples:", len(unlabeled_samples) + len(used_indices))
print("Remaining unlabeled samples:", len(unlabeled_samples))

# Add these examples to your validation dataset
validation_samples = validation_samples.map(lambda x: {
    "questions_and_answers": icl_examples
})

# Print examples to verify quality
print("\nSelected examples:")
for example in icl_examples:
    print(f"\nCategory: {example['answer']}")
    print(f"Text: {example['question']}...")  # Print first 200 chars

Number of ICL examples: 4
Labels covered: ['Business', 'Sci/Tech', 'Sports', 'World']
Missing labels: set()
Original unlabeled samples: 467
Remaining unlabeled samples: 463

Selected examples:

Category: Sports
Text: Expectations Low for Georgia Basketball (AP) AP - Georgia is likely to have a dismal season on the basketball court....

Category: Business
Text: Company: Ameritrade Hldg Corp New With presidential election-related uncertainty safely in the past, retail investors have returned to the US stock market with gusto, and they #39;re likely to stay engaged, several trading companies said Friday....

Category: World
Text: For Arafat, Oslo Remained Symbol of Hope (AP) AP - Norway's capital could not have been further removed from the chaos and bloodshed of the Middle East. Yet it was as a result of top-secret meetings here that two veteran warriors decided it was time to talk peace....

Category: Sci/Tech
Text: Red Hat exec takes Sun to task on open source A top Red Hat executive h

Now let's try the annotation pipeline with the new validation dataset.

In [30]:
gen_data = annotation_pipe.generate(validation_samples)
# Calculate basic accuracy
true_labels = list(map(lambda x: labels[int(x['label'])], gen_data))
pred_labels = list(map(lambda x: x['output'], gen_data))
accuracy = accuracy_score(true_labels, pred_labels)
print(f"Accuracy: {accuracy:.2%}")

Accuracy: 73.33%


Now we have a *much* better accuracy. We can now merge all of the components of the prompt and the ICL examples into a final input dataset for annotation, on the `unlabeled_samples` dataset.

In [31]:
#merge all the components of the prompt and the ICL examples into a final input dataset for annotation
unlabeled_samples = unlabeled_samples.map(lambda x: {
    "system_prompt": system_prompt,
    "simple_task_description": simple_task_description,
    "principles": principles,
    "questions_and_answers": icl_examples,
})
unlabeled_samples

#make sure the features are correct and are same as the validation_samples features
assert unlabeled_samples.features == validation_samples.features, "Features are not the same"


## Annotating the unlabeled dataset

In [90]:
gen_data = annotation_pipe.generate(unlabeled_samples)

Saving results to HuggingFace dataset format

In [91]:
# First rename the columns
gen_data = gen_data.rename_column('label', 'true_label')
gen_data = gen_data.rename_column('output', 'predicted_label')

# Convert numeric labels to string labels if needed
gen_data = gen_data.map(lambda x: {'true_label': labels[int(x['true_label'])]})

# Save to JSONL format
gen_data.to_json('annotation_results.jsonl', lines=True, orient='records')

Creating json from Arrow format: 100%|██████████| 1/1 [00:00<00:00, 130.71ba/s]


882918

## Calculate metrics

In [92]:
# Import necessary libraries
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
from sklearn.metrics import confusion_matrix, classification_report
import json

# Load predictions and true labels
true_labels = []
pred_labels = []

with open('annotation_results.jsonl', 'r') as f:
    for line in f:
        data = json.loads(line)
        true_labels.append(data['true_label'])
        pred_labels.append(data['predicted_label'])

# Calculate basic accuracy
accuracy = accuracy_score(true_labels, pred_labels)
print(f"Accuracy: {accuracy:.2%}")

# Calculate precision, recall, and F1 score for each class
precision, recall, f1, support = precision_recall_fscore_support(true_labels, pred_labels, average=None, labels=labels)

# Print metrics for each class
print("\nPer-class Metrics:")
print("Class\t\tPrecision\tRecall\t\tF1\t\tSupport")
print("-" * 70)
for i, label in enumerate(labels):
    print(f"{label:<12}\t{precision[i]:.2f}\t\t{recall[i]:.2f}\t\t{f1[i]:.2f}\t\t{support[i]}")

# Calculate and print macro and weighted averages
macro_precision, macro_recall, macro_f1, _ = precision_recall_fscore_support(true_labels, pred_labels, average='macro')
weighted_precision, weighted_recall, weighted_f1, _ = precision_recall_fscore_support(true_labels, pred_labels, average='weighted')

print("\nOverall Metrics:")
print(f"Macro Avg:\t{macro_precision:.2f}\t\t{macro_recall:.2f}\t\t{macro_f1:.2f}")
print(f"Weighted Avg:\t{weighted_precision:.2f}\t\t{weighted_recall:.2f}\t\t{weighted_f1:.2f}")

# Print detailed classification report
print("\nDetailed Classification Report:")
print(classification_report(true_labels, pred_labels))

# Create and print confusion matrix
cm = confusion_matrix(true_labels, pred_labels, labels=labels)
print("\nConfusion Matrix:")
print("Labels:", labels)
print(cm)

Accuracy: 78.62%

Per-class Metrics:
Class		Precision	Recall		F1		Support
----------------------------------------------------------------------
World       	0.91		0.67		0.77		106
Sports      	0.87		0.96		0.91		126
Business    	0.77		0.66		0.71		105
Sci/Tech    	0.66		0.82		0.73		126

Overall Metrics:
Macro Avg:	0.80		0.78		0.78
Weighted Avg:	0.80		0.79		0.78

Detailed Classification Report:
              precision    recall  f1-score   support

    Business       0.77      0.66      0.71       105
    Sci/Tech       0.66      0.82      0.73       126
      Sports       0.87      0.96      0.91       126
       World       0.91      0.67      0.77       106

    accuracy                           0.79       463
   macro avg       0.80      0.78      0.78       463
weighted avg       0.80      0.79      0.78       463


Confusion Matrix:
Labels: ['World', 'Sports', 'Business', 'Sci/Tech']
[[ 71  10   6  19]
 [  1 121   4   0]
 [  0   2  69  34]
 [  6   6  11 103]]


The results are much better now compared to when we started. Looking at the confusion matrix, the model is still struggling with the Business/Sci-Tech categories and our prompting strategy can be further improved to help the model learn to annotate these categories better. Some strategies include:

- Adding more specific principles for the Business/Sci-Tech categories
- Adding more detailed examples for the Business/Sci-Tech categories
- Adding hard examples (examples that are close to the decision boundary) to the ICL examples
- Adding more specific system prompt for the Business/Sci-Tech categories
- Adding a description for each of the categories in the generation or introduction component of the prompt


## Conclusion

In this exercise:
- We demonstrated how to build and use custom, composable pipelines with SDG
- We demonstrated how to use SDG to annotate a dataset with a custom annotation pipeline
- We demonstrated the basics of prompt engineering to improve the quality of the annotation
- We learned the importance of using a diverse set of examples in the ICL examples to improve the quality of the annotation
- We learned how to use task specific metrics to understand the performance the prompt against the model for the specific task